In [1]:
import pandas as pd

In [11]:
# import dataset
df = pd.read_csv(r'C:\Users\kettin\Documents\Lomba\GEMASTIKSciPaper\Dataset\ENSO_dataset_balanced_transformer2.csv')
print(df.head())

   feat_0  feat_1    feat_2    feat_3    feat_4    feat_5    feat_6    feat_7  \
0  1950.0     1.0 -1.143366 -1.265137 -1.278335 -1.175420 -0.921329 -0.604104   
1  1950.0     2.0 -1.044837 -1.166932 -1.236573 -1.226775 -1.104201 -0.925912   
2  1950.0     3.0 -1.623563 -1.796104 -1.800000 -1.800000 -1.727419 -1.591487   
3  1950.0     4.0 -1.800000 -1.800000 -1.800000 -1.800000 -1.800000 -1.800000   
4  1950.0     5.0 -1.800000 -1.800000 -1.800000 -1.800000 -1.800000 -1.800000   

     feat_8    feat_9  ...  feat_10981  feat_10982  feat_10983  feat_10984  \
0 -0.486660 -0.482734  ...        -1.8        -1.8        -1.8        -1.8   
1 -0.857395 -0.853803  ...        -1.8        -1.8        -1.8        -1.8   
2 -1.554991 -1.570794  ...        -1.8        -1.8        -1.8        -1.8   
3 -1.800000 -1.800000  ...        -1.8        -1.8        -1.8        -1.8   
4 -1.800000 -1.800000  ...        -1.8        -1.8        -1.8        -1.8   

   feat_10985  feat_10986  feat_10987  feat_

In [12]:
# Cari nilai minimum dan maksimum dari seluruh dataset
min_val = df.min().min()
max_val = df.max().max()

print(f"Rentang nilai data: min = {min_val}, max = {max_val}")

# Cek kolom mana saja yang ada di luar 0–1 (kalau ada)
out_of_range = df[(df < 0).any(axis=1) | (df > 1).any(axis=1)]
print(f"Jumlah baris dengan nilai di luar 0–1: {len(out_of_range)}")


Rentang nilai data: min = -1.7999999523162842, max = 2024.0
Jumlah baris dengan nilai di luar 0–1: 1232


In [13]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# df adalah dataset hasil ADASYN
scaler_final = MinMaxScaler()

# Normalisasi semua kolom
df_scaled_final = pd.DataFrame(
    scaler_final.fit_transform(df),
    columns=df.columns
)

# Cek ulang rentang nilai
print("Rentang nilai data:",
      f"min = {df_scaled_final.min().min()}",
      f"max = {df_scaled_final.max().max()}")
print(df_scaled_final.head())
df_scaled_final = df

Rentang nilai data: min = 0.0 max = 1.0000000000000018
   feat_0    feat_1    feat_2    feat_3    feat_4    feat_5    feat_6  \
0     0.0  0.000000  0.502718  0.447368  0.454277  0.507343  0.571510   
1     0.0  0.090909  0.578152  0.529509  0.490645  0.465627  0.452565   
2     0.0  0.181818  0.135080  0.003259  0.000000  0.000000  0.047208   
3     0.0  0.272727  0.000000  0.000000  0.000000  0.000000  0.000000   
4     0.0  0.363636  0.000000  0.000000  0.000000  0.000000  0.000000   

     feat_7    feat_8    feat_9  ...  feat_10981  feat_10982  feat_10983  \
0  0.646229  0.670085  0.681569  ...         0.0         0.0         0.0   
1  0.472333  0.480931  0.489573  ...         0.0         0.0         0.0   
2  0.112675  0.125007  0.118594  ...         0.0         0.0         0.0   
3  0.000000  0.000000  0.000000  ...         0.0         0.0         0.0   
4  0.000000  0.000000  0.000000  ...         0.0         0.0         0.0   

   feat_10984  feat_10985  feat_10986  feat_10987

In [14]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras import layers, models

# --- Data Preparation ---
X = df_scaled_final.drop(columns=['NINO3.4']).values
y = df_scaled_final['NINO3.4'].values

def create_sequences(X, y, window_size=24):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size + 1):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size-1])
    return np.array(X_seq), np.array(y_seq)

window_size = 18
X_seq, y_seq = create_sequences(X, y, window_size)

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=False)

print("Train shape:", X_train.shape, "Val shape:", X_val.shape, "Test shape:", X_test.shape)


# --- Transformer Model with Projection Fix ---
def build_transformer(input_shape, head_size=64, num_heads=2, ff_dim=64, num_transformer_blocks=2, mlp_units=[64], dropout=0.2):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    for _ in range(num_transformer_blocks):
        # Normalization + Multi-Head Attention
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x1, x1)
        x2 = layers.Add()([x, attention])

        # Feed-Forward Block (dimensi disesuaikan)
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        ff = layers.Dense(ff_dim, activation="relu")(x3)
        ff = layers.Dropout(dropout)(ff)

        # Proyeksikan x2 agar match dengan ff
        proj_x2 = layers.Dense(ff_dim)(x2)
        x = layers.Add()([proj_x2, ff])  # dimensi sudah sama

    # Global Pooling + Output
    x = layers.GlobalAveragePooling1D()(x)
    for units in mlp_units:
        x = layers.Dense(units, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1)(x)  # regresi output

    return models.Model(inputs, outputs)

# Build & Compile Model
model = build_transformer(input_shape=X_train.shape[1:])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=5e-4)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr]
)

# Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
nrmse = rmse / (y_test.max() - y_test.min())
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test NRMSE: {nrmse:.4f}")
print(f"Test R²: {r2:.4f}")


Train shape: (680, 18, 10990) Val shape: (292, 18, 10990) Test shape: (243, 18, 10990)
Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 464ms/step - loss: 66049.8359 - root_mean_squared_error: 245.1178 - val_loss: 93.6284 - val_root_mean_squared_error: 9.6762 - learning_rate: 0.0010
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 377ms/step - loss: 928.6312 - root_mean_squared_error: 30.2239 - val_loss: 57.2464 - val_root_mean_squared_error: 7.5661 - learning_rate: 0.0010
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 392ms/step - loss: 140.4532 - root_mean_squared_error: 11.7276 - val_loss: 38.1830 - val_root_mean_squared_error: 6.1792 - learning_rate: 0.0010
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 389ms/step - loss: 47.6301 - root_mean_squared_error: 6.8976 - val_loss: 9.2148 - val_root_mean_squared_error: 3.0356 - learning_rate: 0.0010
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 386ms/step - loss: 52.3664 - root_mean_squared_error: 7.2319 - val_loss: 1.3588 - val_root_mean_squared_error: 1.1657 - le